# HW1 CSE 152 

In [526]:
%matplotlib
import numpy as np
from imageio import imread
from drawLine import drawLine
from clickPoints import clickPoints
from matplotlib import pyplot as plt
import cv2

Using matplotlib backend: TkAgg


Selecting a pair of images to test with

In [489]:
l = imread('blocks1.gif')
r = imread('blocks2.gif')

In [516]:
xl = clickPoints(l, 10, 1, 1)
xr = clickPoints(r, 10, 1, 1)

Function below takes in points from the left and right images with shape: (2, N) to calculate the fundamental matrix

<img src="8pointalg.jpg" width=450>

In [620]:
def computeF(xl, xr):
    
    A = np.stack(( xl[0, :] * xr[0, :],
                   xl[1, :] * xr[0, :],
                   xr[0, :],
                   xl[0, :] * xr[1, :],
                   xl[1, :] * xr[1, :],
                   xr[1, :],
                   xl[0, :],
                   xl[1, :],
                   np.ones(shape=(len(xl[0]),))),
                axis=-1)
            
    (U, D, V) = np.linalg.svd(A, full_matrices=True)
    
    F = V[-1].reshape(3, 3)
    # Enforcing rank 2 constraint by making the smallest singular value 0
    (U, D, V) = np.linalg.svd(F, full_matrices=True)

    D[-1] = 0
    D = np.diag(D)

    F = np.matmul(U, np.matmul(D, V))
    return F

computeF(xr.T, xl.T)

array([[-1.34997355e-06,  7.91910637e-06, -6.83012152e-04],
       [ 9.40740021e-06,  2.62352625e-05, -5.80919403e-03],
       [-1.61252990e-03, -4.52946167e-03,  9.99971334e-01]])

## Drawing the epipolar lines

Function below takes in points and their respective images with shape: (2, N) to calculate their epipolar lines

In [624]:
def epipolar(xl , xr , l , r):
    
    F = computeF(xl.T, xr.T) * 20

    xl = np.hstack((xl, np.ones((10, 1))))
    xr = np.hstack((xr, np.ones((10, 1))))
    
    leftlines = np.array([np.matmul(F, pt) for pt in xl])
    rightlines = np.array([np.matmul(F.T, pt) for pt in xr])
    
    return F, leftlines, rightlines

In [625]:
F, left, right = epipolar(xl, xr, l, r)

In [626]:
if plt:
    plt.clf()
plt.imshow(l)
drawLine(left[1], l, 'yellow')
drawLine(left[3], l, 'cyan')
drawLine(left[5], l, 'green')
drawLine(left[7], l, 'magenta')
drawLine(left[9], l, 'red')

[[  0.         162.18060049]
 [111.01338813 242.        ]]
[[320.         101.19807257]
 [146.45952056 242.        ]]
[[248.7693334   0.       ]
 [136.0968701 242.       ]]
[[  0.         226.27440183]
 [ 69.81695751 242.        ]]
[[320.         149.52548005]
 [154.05320611 242.        ]]


In [627]:
if plt:
    plt.clf()
plt.imshow(r)
drawLine(right[1], r, 'yellow')
drawLine(right[3], r, 'cyan')
drawLine(right[5], r, 'green')
drawLine(right[7], r, 'magenta')
drawLine(right[9], r, 'red')

[[172.24196556   0.        ]
 [134.55572348 242.        ]]
[[  0.         101.45232202]
 [176.64588272   0.        ]]
[[  0.          85.69755663]
 [177.56481498   0.        ]]
[[172.45906976   0.        ]
 [115.85831629 242.        ]]
[[172.85594288   0.        ]
 [ 81.67888506 242.        ]]
